In [26]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from pprint import pprint
import requests
import gmaps
import os

# Import API key
from api_keys import weather_api_key
from api_keys import g_key
#from api_keys import atlas_key
#from api_keys import nyt_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

#range of latitude and longitude for California
lat_range = (32, 42)
lng_range = (-114, -124)

In [11]:
lat_lngs = [] #this is where we pull the coordinates
cities = [] #we are using the coordinates to loop to the list of cities

lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

213

In [12]:
url = 'http://api.openweathermap.org/data/2.5/weather?' 

#build query URL
#"appid" is the application id, this is our API after the url 
#"&q=" is the query - extensions of the URL help determine specific content or action based on the data being delivered
#in this case our query is the city
query_url = url + "appid=" + weather_api_key + "&q=" + city
query_url = f"{url}appid={weather_api_key}&units=imperial&q="
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=f2deb80b0bea75908cebab9ed8582510&units=imperial&q=


In [8]:
city_name = []
latitude = []
longitude = []


x = 0

print('Beginning Data Retrieval')
print("-"*25)
for i in cities:
    try:
        weather_response = requests.get(query_url + i).json() #each city that is in my cities list
        x = x + 1
        print(f'Processing Record {x} | {i}')
        city_name.append(weather_response["name"])
        latitude.append(weather_response["coord"]["lat"])
        longitude.append(weather_response["coord"]["lon"])
    except KeyError:
        print("City not found")

Beginning Data Retrieval
-------------------------
Processing Record 1 | sunrise manor
Processing Record 2 | winnemucca
Processing Record 3 | blythe
Processing Record 4 | fernley
Processing Record 5 | boulder city
Processing Record 6 | fallon
Processing Record 7 | lompoc
Processing Record 8 | goleta
Processing Record 9 | west wendover
Processing Record 10 | susanville
Processing Record 11 | placerville
Processing Record 12 | pahrump
Processing Record 13 | truckee
Processing Record 14 | ukiah
Processing Record 15 | fortuna
Processing Record 16 | elko
Processing Record 17 | pacific grove
Processing Record 18 | south lake tahoe
Processing Record 19 | exeter
Processing Record 20 | sun valley
Processing Record 21 | cedar city
Processing Record 22 | avenal
Processing Record 23 | el centro
Processing Record 24 | morro bay
Processing Record 25 | altamont
Processing Record 26 | coachella
Processing Record 27 | corcoran
Processing Record 28 | oxnard
Processing Record 29 | magalia
Processing Reco

In [9]:
ca_cities_df = pd.DataFrame({
    "City": city_name,
    "Lat": latitude,
    "Lng": longitude
})
ca_cities_df

,City,Lat,Lng
0,Sunrise Manor,36.21,-115.07
1,Winnemucca,40.97,-117.74
2,Blythe,33.61,-114.60
3,Fernley,39.61,-119.25
4,Boulder City,35.98,-114.83
...,...,...,...
205,Paradise,36.10,-115.15
206,La Cañada Flintridge,34.20,-118.19
207,Lathrop,37.82,-121.28
208,Walnut Creek,37.91,-122.06


In [13]:
#export data to csv
ca_cities_df.to_csv('ca_cities_output.csv')


In [17]:
gmaps.configure(api_key=g_key) 

# Store latitude and longitude in locations, and humidity as humidity
locations = ca_cities_df[["Lat", "Lng"]]

In [18]:
wildfires_df = ca_cities_df.loc[:,["City", "Lat", "Lng"]]

wildfires_df["Wildfires"] = ""

# Display the result
wildfires_df.head()

,City,Lat,Lng,Wildfires
0,Sunrise Manor,36.21,-115.07,
1,Winnemucca,40.97,-117.74,
2,Blythe,33.61,-114.60,
3,Fernley,39.61,-119.25,
4,Boulder City,35.98,-114.83,


In [31]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
          #"type" : "wildfire",
          "keyword" : "california fire",
          "radius" : 500,
          "key" : g_key}

In [30]:
for index, row in wildfires_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Wildfires in {city_name} is {results[0]['name']}.")
        wildfires_df.loc[index, "Wildfires"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No wildfire found")
        
    print("------------")

Retrieving Results for Index 0: Sunrise Manor.
No wildfire found
------------
Retrieving Results for Index 1: Winnemucca.
No wildfire found
------------
Retrieving Results for Index 2: Blythe.
Wildfires in Blythe is Riverside County Fire Dept. Station 43.
------------
Retrieving Results for Index 3: Fernley.
No wildfire found
------------
Retrieving Results for Index 4: Boulder City.
Wildfires in Boulder City is Boulder City Hall.
------------
Retrieving Results for Index 5: Fallon.
No wildfire found
------------
Retrieving Results for Index 6: Lompoc.
Wildfires in Lompoc is Lompoc Police Department.
------------
Retrieving Results for Index 7: La Goulette.
No wildfire found
------------
Retrieving Results for Index 8: West Wendover.
No wildfire found
------------
Retrieving Results for Index 9: Susanville.
Wildfires in Susanville is Susanville Fire Department.
------------
Retrieving Results for Index 10: Placerville.
Wildfires in Placerville is El Dorado County Fire Station 25.
-----

No wildfire found
------------
Retrieving Results for Index 95: Oroville.
Wildfires in Oroville is Oroville Fire Department.
------------
Retrieving Results for Index 96: Greenfield.
No wildfire found
------------
Retrieving Results for Index 97: Vista.
Wildfires in Vista is Vista Fire Department.
------------
Retrieving Results for Index 98: San Bruno.
Wildfires in San Bruno is San Bruno Fire Department.
------------
Retrieving Results for Index 99: Clearlake.
Wildfires in Clearlake is Lake County Fire Protection District - Station 70.
------------
Retrieving Results for Index 100: North Las Vegas.
No wildfire found
------------
Retrieving Results for Index 101: Mexicali.
No wildfire found
------------
Retrieving Results for Index 102: Yucaipa.
No wildfire found
------------
Retrieving Results for Index 103: Yucca Valley.
No wildfire found
------------
Retrieving Results for Index 104: Quartz Hill.
Wildfires in Quartz Hill is Los Angeles County Fire Dept. Station 84.
------------
Retr

Wildfires in Crestline is San Bernardino County Fire Station 25.
------------
Retrieving Results for Index 187: Concord.
Wildfires in Concord is Contra Costa County Fire Protection District Station 6.
------------
Retrieving Results for Index 188: Rio Linda.
Wildfires in Rio Linda is Sacramento Metro Fire District Station 111.
------------
Retrieving Results for Index 189: San Marcos.
Wildfires in San Marcos is San Marcos Fire Station.
------------
Retrieving Results for Index 190: Carlsbad.
No wildfire found
------------
Retrieving Results for Index 191: Clayton.
No wildfire found
------------
Retrieving Results for Index 192: Palmdale.
Wildfires in Palmdale is Los Angeles County Fire Dept. Station #37.
------------
Retrieving Results for Index 193: Livingston.
No wildfire found
------------
Retrieving Results for Index 194: Arroyo Grande.
No wildfire found
------------
Retrieving Results for Index 195: Elk Grove.
Wildfires in Elk Grove is Elk Grove Fire Department.
------------
Retri